This is an exploratory notebook for the stanford contracts database. They have worked with the EDGAR database to make it machine readable, so I am going to play around with what is in the folder just to see if it could be useful to this project

In [1]:
import os
import polars as pl
import re
import PyPDF2
import textwrap
from rapidfuzz import process, fuzz

In [2]:
#starting with one quarter of one year for size and so we don't have to traverse the data directory
data_pathway = '/Users/georgia/Desktop/stanford_law/padelson/production_contracts/2000/Q1'

#list all files in the data pathway
files = os.listdir(data_pathway)

#print the number of files
print(len(files))

173


In [3]:
#see the filetypes in this folder so we know how to load them in 
file_extensions = set()
for f in files:
    _, ext = os.path.splitext(f)
    file_extensions.add(ext.lower())

print("File extensions in folder:", file_extensions)

File extensions in folder: {'', '.pdf', '.html'}


In [4]:
metadata = pl.read_csv('/Users/georgia/Desktop/stanford_law/stanford_metadata.csv')

In [5]:
# Display one full row from the metadata DataFrame in a nice, readable way
import pprint

row = metadata.row(0)
columns = metadata.columns
row_dict = dict(zip(columns, row))
pprint.pprint(row_dict, sort_dicts=False)



{'': 0,
 'year': 2000,
 'cik': 1001136,
 'company.name': 'STOCKWALK COM GROUP INC',
 'form.type': '10-Q',
 'date.filed': '2000-02-14',
 'edgar.link': 'edgar/data/1001136/0000950124-00-000624.txt',
 'quarter': 1,
 'index.link': 'edgar/data/1001136/0000950124-00-000624/000095012400000624-index.html',
 'contract.link': '/Archives/edgar/data/1001136/000095012400000624/0000950124-00-000624-d2.html',
 'exhibit': 'EX-2.1',
 'description': 'asset purchase agreement',
 'exhibit_lead': 'EX-2',
 'contract': '/Archives/edgar/data/1001136/000095012400000624/0000950124-00-000624-d2.html',
 'type_label': 'LABEL_4',
 'type_score': 0.9999955892562866,
 'amend': 0,
 'restate': 0,
 'joinder': 0,
 'termination': 0,
 'parties': "['stockwalkcom inc', 'stockwalkcom group inc', 'stockwalk com "
            "group inc', 'william yang', 'm-one investment securities inc']",
 'agreement_type': 'purchase&ma',
 'parties_cleaned': "['m-one investment securities inc', 'william yang', "
                    "'stockwalk

In [6]:
def read_htm_file(filepath):
    """Read an .htm file and return its text content (stripped of HTML tags)."""
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        html = f.read()
    # Remove HTML tags for plain text extraction
    text = re.sub('<[^<]+?>', '', html)
    return text.strip()


In [7]:
def read_pdf_file(filepath):
    """Read a .pdf file and return its text content."""
    text = ""
    with open(filepath, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text.strip()


In [8]:
#get the contracts themselves into a dataframe
records = []
for fname in files:
    fpath = os.path.join(data_pathway, fname)
    ext = os.path.splitext(fname)[1].lower()
    if ext in ['.htm', '.html']:
        try:
            text = read_htm_file(fpath)
        except Exception as e:
            text = f"[ERROR reading HTML: {e}]"
    elif ext == '.pdf':
        try:
            text = read_pdf_file(fpath)
        except Exception as e:
            text = f"[ERROR reading PDF: {e}]"
    else:
        text = "[UNSUPPORTED FILETYPE]"
    records.append({'title': fname, 'text': text})

# Create polars DataFrame and explicitly set column dtypes to avoid 'str' header confusion
contracts_df = pl.DataFrame(records, schema={"title": pl.String, "text": pl.String})

In [9]:
contracts_df

title,text
str,str
"""823534..000082353400000004..00…","""EX10.14 EXHIBIT 10.14 EMPL…"
"""39273..000003927300000005..000…","""EXHIBIT 10.1 &nbsp;&nbsp;&nbs…"
"""929455..000092945500000002..00…","""EXHIBIT 10.20 NEITHER THIS WAR…"
"""716399..000091205700004404..00…","""Prepared by MERRILL CORPORATIO…"
"""731190..000095013200000238..00…","""FORM OF BONUS AGREEMENT Ex…"
…,…
"""1089786..000095014400001606..0…","""Lanier Worldwide, Inc. / Exhib…"
"""731190..000095013200000238..00…","""PHILIP STRAWBRIDGE CONTRACT TE…"
"""912093..000091209300000004..00…","""Merger AGREEMENT AND PLAN…"


In [10]:
# Select a row index to display (e.g., the first row)
row_idx = 0

# Get the title and text
row = contracts_df.row(row_idx)
title, text = row[0], row[1]

print(f"Title: {title}\n")
print("Text:\n")
print(textwrap.fill(text, width=100))


Title: 823534..000082353400000004..0000823534-00-000004-d7.html

Text:

EX10.14     EXHIBIT 10.14 EMPLOYMENT AGREEMENT   MEMORANDUM OF AGREEMENT entered into at Stamford,
Connecticut, this 1st day of October, 1999.     BY AND BETWEEN:  REPAP ENTERPRISES INC., a company
duly incorporated under the laws of Canada, having its executive offices at 300 Atlantic Street,
Suite 200, Stamford, Connecticut, 06901, herein acting and represented by Stephen C. Larson, duly
authorized to act hereunder for the purposes of the present Agreement as he so declares;   &nbsp;
&nbsp;   &nbsp;  (hereinafter the "Corporation")   &nbsp;  &nbsp;   AND  TERRY W. McBRIDE, business
executive, having his address for the purposes of the present Agreement at 189 Brookwood Lane, New
Canaan, Connecticut 06840   &nbsp;  &nbsp;   &nbsp;  (hereinafter the "Executive")
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;THE PARTIES DECLARE AS FOLLOWS: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;WHEREAS
the Corporation wishes to enlist the Executive's servic

## Combining meta and full data

In [11]:
# To combine contracts_df and metadata, extract the substring after the second '..' in contracts_df['title'],
# then check if this substring appears anywhere in metadata['contract'].
# We'll create a new column in contracts_df for this extracted sequence, then perform a join based on substring matching.

def extract_short_title(s):
    # Extract substring after the second occurrence of '..'
    parts = s.split('..', 2)
    if len(parts) > 2:
        return parts[2]
    return ""

In [12]:
extract_short_title(contracts_df['title'][0])

'0000823534-00-000004-d7.html'

In [13]:
contracts_df = contracts_df.with_columns(
    pl.Series([extract_short_title(s) for s in contracts_df["title"]]).alias("short_title")
)

In [14]:
def find_matching_contract(short_title, contracts):
    for idx, contract in enumerate(contracts):
        if short_title and short_title in contract:
            return idx
    return None

In [15]:
# Get the list of contracts from metadata
metadata_contracts = metadata["contract"].to_list()

def match_titles(metadata_contracts):
    # For each row in contracts_df, find the matching index in metadata
    matching_indices = [
        find_matching_contract(short_title, metadata_contracts)
        for short_title in contracts_df["short_title"]
    ]
        
    # Track if any warning was printed
    any_warning = False
    for title, idx in zip(contracts_df["title"], matching_indices):
        if idx is None:
            print(f"WARNING: No matching index found for title: {title}")
            any_warning = True
        elif isinstance(idx, list) and len(idx) != 1:
            print(f"WARNING: Multiple ({len(idx)}) matching indices found for title: {title} -> {idx}")
            any_warning = True
        elif isinstance(idx, list) and len(idx) == 1:
            pass  # exactly one match, all good
        elif isinstance(idx, int):
            pass  # exactly one match, all good
        else:
            print(f"WARNING: Unexpected matching index for title: {title} -> {idx}")
            any_warning = True

    if not any_warning:
        print("All contract titles successfully matched to metadata entries!")
    
    return matching_indices
        

In [16]:
#get rid of the .ipynb_checkpoints row of the contracts_df
contracts_df = contracts_df.filter(pl.col("title") != ".ipynb_checkpoints")

#get rid of the .ipynb_checkpoints row of the metadata
metadata = metadata.filter(pl.col("contract") != ".ipynb_checkpoints")

In [17]:
# Use match_titles to get matching indices
matching_indices = match_titles(metadata_contracts)

# Build a DataFrame with the matched metadata rows (or None)
matched_metadata = metadata[matching_indices].with_columns(
    pl.Series(matching_indices).alias("metadata_idx")
)

All contract titles successfully matched to metadata entries!


In [18]:
# Concatenate contracts_df and matched_metadata horizontally
df_combined = contracts_df.with_columns(
    matched_metadata
)

In [19]:
df_combined

title,text,short_title,,year,cik,company.name,form.type,date.filed,edgar.link,quarter,index.link,contract.link,exhibit,description,exhibit_lead,contract,type_label,type_score,amend,restate,joinder,termination,parties,agreement_type,parties_cleaned,master_parties,metadata_idx
str,str,str,i64,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64
"""823534..000082353400000004..00…","""EX10.14 EXHIBIT 10.14 EMPL…","""0000823534-00-000004-d7.html""",145,2000,823534,"""REPAP ENTERPRISES INC""","""10-K""","""2000-03-29""","""edgar/data/823534/0000823534-0…",1,"""edgar/data/823534/0000823534-0…","""/Archives/edgar/data/823534/00…","""EX-10""",null,"""EX-10""","""/Archives/edgar/data/823534/00…","""LABEL_1""",0.999994,0,0,0,0,"""['repap enterprises inc', 'ter…","""employment""","""['repap enterprises inc', 'ste…","""['repap enterprises inc', 'ste…",145
"""39273..000003927300000005..000…","""EXHIBIT 10.1 &nbsp;&nbsp;&nbs…","""0000039273-00-000005-d2.html""",67,2000,39273,"""FROZEN FOOD EXPRESS INDUSTRIES…","""8-K""","""2000-01-07""","""edgar/data/39273/0000039273-00…",1,"""edgar/data/39273/0000039273-00…","""/Archives/edgar/data/39273/000…","""EX-10""",null,"""EX-10""","""/Archives/edgar/data/39273/000…","""LABEL_0""",0.999997,1,0,0,0,"""['frozen food express inc', 'w…","""security""","""['w andb', 'conwell corporatio…","""['w andb', 'conwell corporatio…",67
"""929455..000092945500000002..00…","""EXHIBIT 10.20 NEITHER THIS WAR…","""0000929455-00-000002-d14.pdf""",175,2000,929455,"""CLUBCORP INC""","""10-K""","""2000-03-24""","""edgar/data/929455/0000929455-0…",1,"""edgar/data/929455/0000929455-0…","""/Archives/edgar/data/929455/00…","""EX-10.20""",null,"""EX-10""","""/Archives/edgar/data/929455/00…","""LABEL_7""",0.445939,0,0,0,0,"""['clubcorp inc']""","""na""","""['clubcorp inc']""","""['clubcorp inc']""",175
"""716399..000091205700004404..00…","""Prepared by MERRILL CORPORATIO…","""0000912057-00-004404-d2.html""",78,2000,716399,"""CALENDAR CAPITAL INC""","""8-K""","""2000-02-08""","""edgar/data/716399/0000912057-0…",1,"""edgar/data/716399/0000912057-0…","""/Archives/edgar/data/716399/00…","""EX-10.1""","""exhibit 10.1""","""EX-10""","""/Archives/edgar/data/716399/00…","""LABEL_5""",0.664164,0,0,0,0,"""['cc', 'calendar capital inc',…","""shareholder""","""['calendar capital inc', 'entr…","""['calendar capital inc', 'entr…",78
"""731190..000095013200000238..00…","""FORM OF BONUS AGREEMENT Ex…","""0000950132-00-000238-d12.html""",107,2000,731190,"""IT GROUP INC""","""10-K""","""2000-03-30""","""edgar/data/731190/0000950132-0…",1,"""edgar/data/731190/0000950132-0…","""/Archives/edgar/data/731190/00…","""EX-10.(III)48""","""form of bonus agreement""","""EX-10""","""/Archives/edgar/data/731190/00…","""LABEL_1""",1.0,0,0,0,0,"""['it group inc', 'the it group…","""employment""","""['the it group inc', 'it group…","""['it group inc']""",107
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1089786..000095014400001606..0…","""Lanier Worldwide, Inc. / Exhib…","""0000950144-00-001606-d3.html""",29,2000,1089786,"""LANIER WORLDWIDE INC""","""10-Q""","""2000-02-08""","""edgar/data/1089786/0000950144-…",1,"""edgar/data/1089786/0000950144-…","""/Archives/edgar/data/1089786/0…","""EX-10.2""","""save to accumulate retirement …","""EX-10""","""/Archives/edgar/data/1089786/0…","""LABEL_1""",0.999996,1,1,0,0,"""['lanier worldwide inc']""","""employment""","""['lanier worldwide inc']""","""['lanier worldwide inc']""",29
"""731190..000095013200000238..00…","""PHILIP STRAWBRIDGE CONTRACT TE…","""0000950132-00-000238-d11.html""",106,2000,731190,"""IT GROUP INC""","""10-K""","""2000-03-30""","""edgar/data/731190/0000950132-0…",1,"""edgar/data/731190/0000950132-0…","""/Archives/edgar/data/731190/00…","""EX-10.(III)40""","""philip strawbridge contract te…","""EX-10""","""/Archives/edgar/data/731190/00…","""LABEL_1""",1.0,0,0,0,1,"""['it group inc', 'philip straw…","""employment""","""['philip strawbridge', 'it co

## Matching these companies with their headquarters

In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("peopledatalabssf/free-7-million-company-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/georgia/.cache/kagglehub/datasets/peopledatalabssf/free-7-million-company-dataset/versions/1


In [21]:
#load in the kaggle dataset
local_path = '/Users/georgia/.cache/kagglehub/datasets/peopledatalabssf/free-7-million-company-dataset/versions/1/companies_sorted.csv'
companies_df = pl.read_csv(local_path)


In [22]:
companies_df

,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
i64,str,str,f64,str,str,str,str,str,i64,i64
5872184,"""ibm""","""ibm.com""",1911.0,"""information technology and ser…","""10001+""","""new york, new york, united sta…","""united states""","""linkedin.com/company/ibm""",274047,716906
4425416,"""tata consultancy services""","""tcs.com""",1968.0,"""information technology and ser…","""10001+""","""bombay, maharashtra, india""","""india""","""linkedin.com/company/tata-cons…",190771,341369
21074,"""accenture""","""accenture.com""",1989.0,"""information technology and ser…","""10001+""","""dublin, dublin, ireland""","""ireland""","""linkedin.com/company/accenture""",190689,455768
2309813,"""us army""","""goarmy.com""",1800.0,"""military""","""10001+""","""alexandria, virginia, united s…","""united states""","""linkedin.com/company/us-army""",162163,445958
1558607,"""ey""","""ey.com""",1989.0,"""accounting""","""10001+""","""london, greater london, united…","""united kingdom""","""linkedin.com/company/ernstandy…",158363,428960
…,…,…,…,…,…,…,…,…,…,…
1494427,"""certiport vouchers""","""certiportvouchers.com""",2011.0,"""information technology and ser…","""1 - 10""",null,null,"""linkedin.com/company/certiport…",0,1
1494429,"""black tiger fight club""","""blacktigerclub.com""",2006.0,"""health, wellness and fitness""","""1 - 10""","""peking, beijing, china""","""china""","""linkedin.com/company/black-tig…",0,6
4768462,"""catholic bishop of chicago""",null,null,"""religious institutions""","""1 - 10""","""inverness, illinois, united st…","""united states""","""linkedin.com/company/catholic-…",0,1


In [23]:
# See how many master_parties in df_combined have a fuzzy match with any company in companies_df["name"]

# Get unique company names and master parties, dropping nulls
company_names = companies_df["name"].unique().drop_nulls().to_list()
# Flatten the list of lists in master_parties, drop nulls, and get unique
master_parties = (
    pl.Series([p for sublist in df_combined["master_parties"].drop_nulls().to_list() for p in (sublist if isinstance(sublist, list) else [sublist])])
    .unique()
    .drop_nulls()
    .to_list()
)
print(len(master_parties))
print(master_parties)

137
["['centra software inc']", "['com inc', 'michael t devlin', 'rational software corporation', 'devclick']", "['dakota growers pasta co inc', 'american community bank', 'alta california bank', 'atlantic capital bancshares inc', 'american community bancshares inc', 'acb']", "['clubcorp inc']", "['w andb', 'conwell corporation', 'ffe transportation services inc', 'w and b refrigeration service company', 'ffe inc', 'frozen food express industries inc', 'frozen food express inc', 'lisa motor lines inc']", "['q comm international inc']", "['ensco offshore company', 'ensco international inc', 'secretary of transportation', 'tdi halter l', 'friede goldman offshore texas']", "['rainbow acquisition inc', 'e-tek dynamics inc', 'jds uniphase corporation']", "['north shore gas co il']", "['philip o strawbridge', 'it corporation', 'it group inc']", "['moto photo inc', 'paul pieschel']", "['credit suisse first boston', 'cibc world markets corp one world financial center new york', 'protein design

In [24]:
def search_company_matches(search_name, companies_df, fuzzy_threshold=80):
    # Exact match
    exact_matches = companies_df.filter(pl.col("name") == search_name)
    if exact_matches.height > 0:
        print("Exact match:")
        print(exact_matches)
        print(f"Number of exact matches: {exact_matches.height}")
        return {
            "exact": exact_matches,
            "substring": None,
            "fuzzy": None
        }

    # Substring/contain match (case-insensitive)
    contain_matches = companies_df.filter(
        pl.col("name").str.to_lowercase().str.contains(search_name.lower())
    )
    if contain_matches.height > 0:
        print("\nContain (substring) match:")
        print(contain_matches)
        print(f"Number of substring matches: {contain_matches.height}")
        return {
            "exact": None,
            "substring": contain_matches,
            "fuzzy": None
        }

    # Fuzzy match (using token_sort_ratio)
    company_names = companies_df["name"].unique().drop_nulls().to_list()
    fuzzy_match, score, idx = process.extractOne(search_name, company_names, scorer=fuzz.token_sort_ratio)
    print(f"\nBest fuzzy match: {fuzzy_match} (score: {score})")
    fuzzy_matches_df = None
    fuzzy_count = 0
    if score >= fuzzy_threshold:
        fuzzy_matches_df = companies_df.filter(pl.col("name") == fuzzy_match)
        print(fuzzy_matches_df)
        fuzzy_count = fuzzy_matches_df.height
    print(f"Number of fuzzy matches (above threshold): {fuzzy_count}")

    return {
        "exact": None,
        "substring": None,
        "fuzzy": {
            "match": fuzzy_match,
            "score": score,
            "df": fuzzy_matches_df
        }
    }

In [25]:
result = search_company_matches("fifth third", companies_df)

Exact match:
shape: (1, 11)
┌─────────┬───────┬────────┬─────────┬───┬─────────┬──────────────────┬──────────┬─────────────────┐
│         ┆ name  ┆ domain ┆ year    ┆ … ┆ country ┆ linkedin url     ┆ current  ┆ total employee  │
│ ---     ┆ ---   ┆ ---    ┆ founded ┆   ┆ ---     ┆ ---              ┆ employee ┆ estimate        │
│ i64     ┆ str   ┆ str    ┆ ---     ┆   ┆ str     ┆ str              ┆ estimate ┆ ---             │
│         ┆       ┆        ┆ f64     ┆   ┆         ┆                  ┆ ---      ┆ i64             │
│         ┆       ┆        ┆         ┆   ┆         ┆                  ┆ i64      ┆                 │
╞═════════╪═══════╪════════╪═════════╪═══╪═════════╪══════════════════╪══════════╪═════════════════╡
│ 2291055 ┆ fifth ┆ null   ┆ 1858.0  ┆ … ┆ united  ┆ linkedin.com/com ┆ 43       ┆ 63              │
│         ┆ third ┆        ┆         ┆   ┆ states  ┆ pany/fifth-thi…  ┆          ┆                 │
└─────────┴───────┴────────┴─────────┴───┴─────────┴───────────

In [26]:
_ = search_company_matches("zamba corp", companies_df)


Contain (substring) match:
shape: (1, 11)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│         ┆ name       ┆ domain    ┆ year      ┆ … ┆ country   ┆ linkedin  ┆ current   ┆ total     │
│ ---     ┆ ---        ┆ ---       ┆ founded   ┆   ┆ ---       ┆ url       ┆ employee  ┆ employee  │
│ i64     ┆ str        ┆ str       ┆ ---       ┆   ┆ str       ┆ ---       ┆ estimate  ┆ estimate  │
│         ┆            ┆           ┆ f64       ┆   ┆           ┆ str       ┆ ---       ┆ ---       │
│         ┆            ┆           ┆           ┆   ┆           ┆           ┆ i64       ┆ i64       │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 5393610 ┆ zamba corp ┆ zambasolu ┆ null      ┆ … ┆ united    ┆ linkedin. ┆ 0         ┆ 3         │
│         ┆ oration    ┆ tions.com ┆           ┆   ┆ states    ┆ com/compa ┆           ┆           │
│         ┆            ┆           ┆           ┆

In [27]:
_ = search_company_matches("wta campbell technology park", companies_df)


Best fuzzy match: canberra technology park (score: 76.92307692307692)
Number of fuzzy matches (above threshold): 0


In [28]:
df_combined

title,text,short_title,,year,cik,company.name,form.type,date.filed,edgar.link,quarter,index.link,contract.link,exhibit,description,exhibit_lead,contract,type_label,type_score,amend,restate,joinder,termination,parties,agreement_type,parties_cleaned,master_parties,metadata_idx
str,str,str,i64,i64,i64,str,str,str,str,i64,str,str,str,str,str,str,str,f64,i64,i64,i64,i64,str,str,str,str,i64
"""823534..000082353400000004..00…","""EX10.14 EXHIBIT 10.14 EMPL…","""0000823534-00-000004-d7.html""",145,2000,823534,"""REPAP ENTERPRISES INC""","""10-K""","""2000-03-29""","""edgar/data/823534/0000823534-0…",1,"""edgar/data/823534/0000823534-0…","""/Archives/edgar/data/823534/00…","""EX-10""",null,"""EX-10""","""/Archives/edgar/data/823534/00…","""LABEL_1""",0.999994,0,0,0,0,"""['repap enterprises inc', 'ter…","""employment""","""['repap enterprises inc', 'ste…","""['repap enterprises inc', 'ste…",145
"""39273..000003927300000005..000…","""EXHIBIT 10.1 &nbsp;&nbsp;&nbs…","""0000039273-00-000005-d2.html""",67,2000,39273,"""FROZEN FOOD EXPRESS INDUSTRIES…","""8-K""","""2000-01-07""","""edgar/data/39273/0000039273-00…",1,"""edgar/data/39273/0000039273-00…","""/Archives/edgar/data/39273/000…","""EX-10""",null,"""EX-10""","""/Archives/edgar/data/39273/000…","""LABEL_0""",0.999997,1,0,0,0,"""['frozen food express inc', 'w…","""security""","""['w andb', 'conwell corporatio…","""['w andb', 'conwell corporatio…",67
"""929455..000092945500000002..00…","""EXHIBIT 10.20 NEITHER THIS WAR…","""0000929455-00-000002-d14.pdf""",175,2000,929455,"""CLUBCORP INC""","""10-K""","""2000-03-24""","""edgar/data/929455/0000929455-0…",1,"""edgar/data/929455/0000929455-0…","""/Archives/edgar/data/929455/00…","""EX-10.20""",null,"""EX-10""","""/Archives/edgar/data/929455/00…","""LABEL_7""",0.445939,0,0,0,0,"""['clubcorp inc']""","""na""","""['clubcorp inc']""","""['clubcorp inc']""",175
"""716399..000091205700004404..00…","""Prepared by MERRILL CORPORATIO…","""0000912057-00-004404-d2.html""",78,2000,716399,"""CALENDAR CAPITAL INC""","""8-K""","""2000-02-08""","""edgar/data/716399/0000912057-0…",1,"""edgar/data/716399/0000912057-0…","""/Archives/edgar/data/716399/00…","""EX-10.1""","""exhibit 10.1""","""EX-10""","""/Archives/edgar/data/716399/00…","""LABEL_5""",0.664164,0,0,0,0,"""['cc', 'calendar capital inc',…","""shareholder""","""['calendar capital inc', 'entr…","""['calendar capital inc', 'entr…",78
"""731190..000095013200000238..00…","""FORM OF BONUS AGREEMENT Ex…","""0000950132-00-000238-d12.html""",107,2000,731190,"""IT GROUP INC""","""10-K""","""2000-03-30""","""edgar/data/731190/0000950132-0…",1,"""edgar/data/731190/0000950132-0…","""/Archives/edgar/data/731190/00…","""EX-10.(III)48""","""form of bonus agreement""","""EX-10""","""/Archives/edgar/data/731190/00…","""LABEL_1""",1.0,0,0,0,0,"""['it group inc', 'the it group…","""employment""","""['the it group inc', 'it group…","""['it group inc']""",107
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1089786..000095014400001606..0…","""Lanier Worldwide, Inc. / Exhib…","""0000950144-00-001606-d3.html""",29,2000,1089786,"""LANIER WORLDWIDE INC""","""10-Q""","""2000-02-08""","""edgar/data/1089786/0000950144-…",1,"""edgar/data/1089786/0000950144-…","""/Archives/edgar/data/1089786/0…","""EX-10.2""","""save to accumulate retirement …","""EX-10""","""/Archives/edgar/data/1089786/0…","""LABEL_1""",0.999996,1,1,0,0,"""['lanier worldwide inc']""","""employment""","""['lanier worldwide inc']""","""['lanier worldwide inc']""",29
"""731190..000095013200000238..00…","""PHILIP STRAWBRIDGE CONTRACT TE…","""0000950132-00-000238-d11.html""",106,2000,731190,"""IT GROUP INC""","""10-K""","""2000-03-30""","""edgar/data/731190/0000950132-0…",1,"""edgar/data/731190/0000950132-0…","""/Archives/edgar/data/731190/00…","""EX-10.(III)40""","""philip strawbridge contract te…","""EX-10""","""/Archives/edgar/data/731190/00…","""LABEL_1""",1.0,0,0,0,1,"""['it group inc', 'philip straw…","""employment""","""['philip strawbridge', 'it co

In [29]:
import ast

# master_parties is a list of strings like "['bristol hotels and resorts', 'jeffrey p mayer']"
# We want to flatten it into a list of all entities, splitting on commas inside the lists

flat_master_parties = []
for entry in master_parties:
    # Safely evaluate the string to a Python list
    try:
        entities = ast.literal_eval(entry)
        if isinstance(entities, list):
            flat_master_parties.extend([e.strip() for e in entities])
        else:
            # If not a list, just add the string itself
            flat_master_parties.append(str(entities).strip())
    except Exception:
        # If parsing fails, fallback to splitting on comma
        flat_master_parties.extend([e.strip(" '") for e in entry.split(",") if e.strip(" '")])

flat_master_parties

['centra software inc',
 'com inc',
 'michael t devlin',
 'rational software corporation',
 'devclick',
 'dakota growers pasta co inc',
 'american community bank',
 'alta california bank',
 'atlantic capital bancshares inc',
 'american community bancshares inc',
 'acb',
 'clubcorp inc',
 'w andb',
 'conwell corporation',
 'ffe transportation services inc',
 'w and b refrigeration service company',
 'ffe inc',
 'frozen food express industries inc',
 'frozen food express inc',
 'lisa motor lines inc',
 'q comm international inc',
 'ensco offshore company',
 'ensco international inc',
 'secretary of transportation',
 'tdi halter l',
 'friede goldman offshore texas',
 'rainbow acquisition inc',
 'e-tek dynamics inc',
 'jds uniphase corporation',
 'north shore gas co il',
 'philip o strawbridge',
 'it corporation',
 'it group inc',
 'moto photo inc',
 'paul pieschel',
 'credit suisse first boston',
 'cibc world markets corp one world financial center new york',
 'protein design labs inc de'

In [30]:
len(flat_master_parties)

443

In [31]:
# only keep unique master parties 
flat_master_parties = list(set(flat_master_parties))
len(flat_master_parties)

316

In [32]:
# Commented out because it takes too long to run
# # Go through all names in df_combined and perform matching, storing results

# results = []

# for search_name in flat_master_parties:
#     match_result = search_company_matches(search_name, companies_df)
#     # Try exact match first
#     if match_result["exact"] is not None and match_result["exact"].height > 0:
#         best_match_name = match_result["exact"]["name"][0]
#         score = 100
#     # Then substring match
#     elif match_result["substring"] is not None and match_result["substring"].height > 0:
#         best_match_name = match_result["substring"]["name"][0]
#         score = 100
#     # Then fuzzy match
#     elif match_result["fuzzy"] is not None and match_result["fuzzy"]["score"] >= 0:
#         best_match_name = match_result["fuzzy"]["match"]
#         score = match_result["fuzzy"]["score"]
#     else:
#         best_match_name = "NO MATCH"
#         score = None
#     print(f"Search name: {search_name}, Best match name: {best_match_name}, Score: {score}")
#     results.append({
#         "df_combined_name": search_name,
#         "best_match_name": best_match_name,
#         "fuzzy_score": score
#     })

# # Convert results to a DataFrame
# match_results_df = pl.DataFrame(results)

In [33]:
match_results_df

NameError: name 'match_results_df' is not defined

Now I need to heal the ones that are clearly incorrect one by one

In [ ]:
#first let's split those that have a score of 100. Those should be okay to trust. 
rows_100 = []
for row in match_results_df.iter_rows(named=True):
    if row["fuzzy_score"] == 100:
        rows_100.append(row)
df_100 = pl.DataFrame(rows_100)

In [ ]:
import pandas as pd

pdf = df_100.to_pandas()
pd.set_option("display.max_colwidth", None)  # don't truncate
print(pdf.to_string())


                               df_combined_name                                        best_match_name  fuzzy_score
0                   compaq computer corporation                            compaq computer corporation        100.0
1                                           acb                                                    acb        100.0
2                                           ing                                                    ing        100.0
3                inland real estate corporation                         inland real estate corporation        100.0
4                              service provider                                       service provider        100.0
5                                       com inc        bet (bet networks, a subsidiary of viacom inc.)        100.0
6                                           fda                                                    fda        100.0
7   the prudential insurance company of america  the prudential insuranc

100s that are incorrect: com_inc, scientific technologies inc, michael casey, david adams, etec systems inc, it group inc, govco, trust company national association, ohm corporation, manhattan bank

non_100s that are incorrect: boston acquisition sub inc, us bank national association, merix merix, the inland group inc, calendar capital inc, dakin acquisition corporation, huffy service first inc, john c van leeuwen glens falls national bank, ensco offshore company, american sports design company, william v ehlen, frank m montano, stanlee p greene jr, citibank international plc, michael hammes, jds uniphase corporation, paul d levy, bryan l kelln, tdi halter l, william g gonzalez, w e cantrell, kevin t kabat, valle de oro bank na, selco service, q comm international, frozen food express industries inc, stephen c flaherty, wachovia bank national association, william yang, teltronics inc, mary ellen baker, michael j jandernoa, shoreline financial corporation, john d boyles, hartford steam, louis a dubrow, inland commercial property management inc, dan olson, terry w mcrbide, paul pieschel, dimon incorporated, philip o strawbridge, mdecd, paul levy, gem investments ltd, alan w ott, applied materials inc de, tellabs, bard c r inc, mcms, thomas l hoy, daniel t olson, merrill corporation network, owen s baxter, gregory k daniels, huffy risk management inc, hcac inc, albert c monk, albert t potas, adc telecommunications inc, wayne r koebel, congress financial corporation, john j murphy, blue hill inc, c r bard inc, downtown cogeneration associates, repap enterprises inc, m-one investment securities, royce union bicycle company, paul d levy paul d levy, adobe systems europe limited, gem global yield fund, robert j vosburgh, wells fargo business credit, halpern capital advisors, blue ridge asset funding corporation, inland advisors, roger, paul romanelli, software corporation international, mr ott, gary s bernard, ronald c mishler, road master inc, cibc world markets corp one world financial center new york, stephen capital inc, steven m duffy, stockwalkcom inc, graham, kilroy realty lp, keybank national association, inland real estate advisory services inc, richard c deblasio, glenbrae management services limited, general electric capital corporation, david c schneider, stephen c larson, melvin r heier, donald r britton, w and b refrigeration service company, mississippi department of economic and community development, river bancorp inc, energy east corporation 

In [34]:
possible_companies = companies_df["name"].to_list()
unique_possible_companies = list(set(possible_companies))


In [35]:
#let's try a different approach. Instead of fuzzy matching, I want to give a list of 
# the possible options to an LLM and then see which one they think is the closest match. 
# If it isn't 100% sure, give three options and I can choose

def message(possible_companies, single_company):
    messages = [
        {
            "role": "system",
            "content": "You are an expert in corporate accounting and reporting. We want to match company names between two lists."
        },
        {
            "role": "user",
            "content": f"Here's the list of all the possible companies: {possible_companies}. This is the single company: {single_company}. Please tell me which one of the first companies best matches. If you aren't super confident, you can give me three options. Return your answer in this JSON format: {{\"match\": \"Company Name[s]\", \"confidence\": \"Confidence Level\"}}"}
    ]
    
    return messages


In [36]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"), 
    base_url = "https://openrouter.ai/api/v1"
)

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed with progress bar
all_text_embeddings = []
for company in tqdm(unique_possible_companies, desc="Embedding possible companies"):
    all_text_embeddings.append(model.encode(company, convert_to_tensor=True))

single_company_embeddings = []
for party in tqdm(flat_master_parties, desc="Embedding master parties"):
    single_company_embeddings.append(model.encode(party, convert_to_tensor=True))


Embedding possible companies:   2%|▏         | 165448/7004635 [14:26:03<34083:29:22, 17.94s/it]IOStream.flush timed out


In [ ]:
response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free", 
    messages= messages,
    temperature=0.1,
    max_tokens=1000
)

KeyboardInterrupt: 

In [ ]:
assistant_reply_1 = response.choices[0].message.content